In [1]:
# libraries
import os
import json
import numpy as np
import pandas as pd
from extra_files import helper as hp
from imageio import imwrite, imread
from skimage.transform import resize

In [2]:
# File paths
data_path = '/home/aldo/Documents/data-cic/'
preprocess_path = data_path + 'preprocess_data'

## Training SSD300 trained with vgg16 backbone trained

In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300_vgg_13_bn import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from extra_files.f1_callback import F1_callback as f1_call
%matplotlib inline

Using TensorFlow backend.
/home/aldo/Documents/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


### Parameters (original SSD300 architecture)

In [4]:
## Parameteres needed for ssd_300() and SSDInputEncoder()

img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 1 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

## Create new model with SSD weights

In [5]:
# 1: Build the Keras model.

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)

# 2: Load some weights into the model.

# TODO: Set the path to the weights you want to load.
# weights_path = '../weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal_epoch-113_loss-3.3136_val_loss-3.3338.h5'
# model.load_weights(weights_path, by_name=True)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [6]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
identity_layer (Lambda)         (None, 300, 300, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
input_mean_normalization (Lambd (None, 300, 300, 3)  0           identity_layer[0][0]             
__________________________________________________________________________________________________
input_channel_swap (Lambda)     (None, 300, 300, 3)  0           input_mean_normalization[0][0]   
__________________________________________________________________________________________________
conv1_1 (C

## Data generator for the training

In [7]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
# images_dir = data_path + 'udacity_driving_datasets'
images_dir = data_path + 'pascal_dataset'
# images_dir = data_path + 'images'

# Ground truth
# train_labels_filename = preprocess_path + '/udc.csv'
# train_labels_filename = preprocess_path + '/cic_pascal_train.csv'
# val_labels_filename   = preprocess_path + '/cic_pascal_val.csv'
train_labels_filename = preprocess_path + '/pascal_train.csv'
val_labels_filename   = preprocess_path + '/pascal_val.csv'
# train_labels_filename = preprocess_path + '/cic_train.csv'
# val_labels_filename   = preprocess_path + '/cic_val.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Optional: Convert the dataset into an HDF5 dataset. This will require more disk space, but will
# speed up the training. Doing this is not relevant in case you activated the `load_images_into_memory`
# option in the constructor, because in that cas the images are in memory already anyway. If you don't
# want to create HDF5 datasets, comment out the subsequent two function calls.

# train_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_train.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_val.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Loading images into memory: 100%|██████████| 2097/2097 [00:08<00:00, 257.64it/s]
Number of images in the training dataset:	  6469
Number of images in the validation dataset:	  2097


In [8]:
# 3: Set the batch size.
batch_size = 20 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	  6469
Number of images in the validation dataset:	  2097


## Remaining training parameters

In [9]:
label_val = np.load('../data-cic/preprocess_data/label_val.npy')
val_images_300 = np.load('../data-cic/preprocess_data/images_val_300x300.npy')

In [10]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)

csv_logger = CSVLogger(filename=data_path + 'history/ssd300_adam_vgg13_bn/ssd300_vgg_13_pascal.csv',
                       separator=',',
                       append=True)

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=11,
                               verbose=1)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.2,
                                         patience=9,
                                         verbose=1,
                                         min_delta=0.001,
                                         cooldown=0,
                                         min_lr=0.00001)

f1_callback = f1_call(0.20, 
                      0.45, 
                      200, 
                      normalize_coords, 
                      img_height, 
                      img_width, 
                      val_images_300,
                      label_val, 
                      (1, 8732, 14),
                      data_path + 'history/ssd300_adam_vgg13_bn/ssd300_vgg_13_pascal_f1.csv')

# learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule,
#                                                 verbose=1)

# terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             early_stopping,
             reduce_learning_rate,
             f1_callback]

In [11]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 1000
steps_per_epoch = 650

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Epoch 1/1000
650/650 [==============================] - 436s 671ms/step - loss: 12.0438 - val_loss: 8.6381

Epoch 00001: val_loss improved from inf to 8.63811, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal.h5
Number of images: 45
Presicion: 0.0488
Recall: 0.4162
F1 score: 0.0874
Improve F1 score from -inf to 0.08739253413176518
Epoch 2/1000
650/650 [==============================] - 420s 647ms/step - loss: 7.6790 - val_loss: 7.3165

Epoch 00002: val_loss improved from 8.63811 to 7.31655, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal.h5
Number of images: 45
Presicion: 0.0485
Recall: 0.386
F1 score: 0.0862
Epoch 3/1000
650/650 [==============================] - 419s 644ms/step - loss: 6.3990 - val_loss: 6.0336

Epoch 00003: val_loss improved from 7.31655 to 6.03360, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal.h5
Number of images: 45
Pr

650/650 [==============================] - 418s 643ms/step - loss: 3.9305 - val_loss: 3.9953

Epoch 00029: val_loss improved from 3.99901 to 3.99535, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal.h5
Number of images: 45
Presicion: 0.082
Recall: 0.4772
F1 score: 0.1399
Epoch 30/1000
650/650 [==============================] - 418s 643ms/step - loss: 3.9008 - val_loss: 3.9112

Epoch 00030: val_loss improved from 3.99535 to 3.91125, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal.h5
Number of images: 45
Presicion: 0.2145
Recall: 0.47
F1 score: 0.2946
Improve F1 score from 0.18152754237063953 to 0.29460174346419304
Epoch 31/1000
650/650 [==============================] - 415s 638ms/step - loss: 3.8851 - val_loss: 3.9273

Epoch 00031: val_loss did not improve from 3.91125
Number of images: 45
Presicion: 0.1922
Recall: 0.5039
F1 score: 0.2782
Epoch 32/1000
650/650 [=============================


Epoch 00055: val_loss did not improve from 3.50887
Number of images: 45
Presicion: 0.503
Recall: 0.3589
F1 score: 0.4189
Epoch 56/1000
650/650 [==============================] - 419s 644ms/step - loss: 3.4597 - val_loss: 3.5400

Epoch 00056: val_loss did not improve from 3.50887
Number of images: 45
Presicion: 0.4792
Recall: 0.355
F1 score: 0.4078
Epoch 57/1000
650/650 [==============================] - 413s 635ms/step - loss: 3.4631 - val_loss: 3.4771

Epoch 00057: val_loss improved from 3.50887 to 3.47710, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal.h5
Number of images: 45
Presicion: 0.5433
Recall: 0.3324
F1 score: 0.4125
Epoch 58/1000
650/650 [==============================] - 419s 644ms/step - loss: 3.4743 - val_loss: 3.5195

Epoch 00058: val_loss did not improve from 3.47710
Number of images: 45
Presicion: 0.5577
Recall: 0.3319
F1 score: 0.4162
Epoch 59/1000
650/650 [==============================] - 419s 644ms/step - loss: 3.4

650/650 [==============================] - 417s 641ms/step - loss: 3.3205 - val_loss: 3.4609

Epoch 00086: val_loss did not improve from 3.37041
Number of images: 45
Presicion: 0.4779
Recall: 0.4098
F1 score: 0.4412
Epoch 87/1000
650/650 [==============================] - 417s 641ms/step - loss: 3.3174 - val_loss: 3.4012

Epoch 00087: val_loss did not improve from 3.37041
Number of images: 45
Presicion: 0.5663
Recall: 0.4166
F1 score: 0.4801
Improve F1 score from 0.4794591119212615 to 0.48009352401736527
Epoch 88/1000
650/650 [==============================] - 418s 644ms/step - loss: 3.3041 - val_loss: 3.4545

Epoch 00088: val_loss did not improve from 3.37041
Number of images: 45
Presicion: 0.4774
Recall: 0.4008
F1 score: 0.4358
Epoch 89/1000
650/650 [==============================] - 418s 644ms/step - loss: 3.3006 - val_loss: 3.4836

Epoch 00089: val_loss did not improve from 3.37041
Number of images: 45
Presicion: 0.4522
Recall: 0.3733
F1 score: 0.409
Epoch 90/1000
650/650 [========

650/650 [==============================] - 419s 645ms/step - loss: 3.0279 - val_loss: 3.1436

Epoch 00115: val_loss did not improve from 3.12805
Number of images: 45
Presicion: 0.5585
Recall: 0.4669
F1 score: 0.5086
Epoch 116/1000
650/650 [==============================] - 419s 645ms/step - loss: 3.0120 - val_loss: 3.1555

Epoch 00116: val_loss did not improve from 3.12805
Number of images: 45
Presicion: 0.5859
Recall: 0.442
F1 score: 0.5039
Epoch 117/1000
650/650 [==============================] - 419s 645ms/step - loss: 3.0071 - val_loss: 3.1393

Epoch 00117: val_loss did not improve from 3.12805
Number of images: 45
Presicion: 0.5743
Recall: 0.4508
F1 score: 0.5051
Epoch 118/1000
650/650 [==============================] - 420s 645ms/step - loss: 2.9959 - val_loss: 3.1402

Epoch 00118: val_loss did not improve from 3.12805
Number of images: 45
Presicion: 0.5533
Recall: 0.4481
F1 score: 0.4952
Epoch 119/1000
650/650 [==============================] - 418s 644ms/step - loss: 2.9870 - v

Number of images: 45
Presicion: 0.5493
Recall: 0.4808
F1 score: 0.5128
Epoch 147/1000
650/650 [==============================] - 422s 650ms/step - loss: 2.9278 - val_loss: 3.0692

Epoch 00147: val_loss improved from 3.07187 to 3.06920, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal.h5
Number of images: 45
Presicion: 0.567
Recall: 0.4797
F1 score: 0.5197
Epoch 148/1000
650/650 [==============================] - 418s 642ms/step - loss: 2.9059 - val_loss: 3.0755

Epoch 00148: val_loss did not improve from 3.06920
Number of images: 45
Presicion: 0.5546
Recall: 0.4687
F1 score: 0.508
Epoch 149/1000
650/650 [==============================] - 413s 635ms/step - loss: 2.9061 - val_loss: 3.0743

Epoch 00149: val_loss did not improve from 3.06920
Number of images: 45
Presicion: 0.5646
Recall: 0.4692
F1 score: 0.5125
Epoch 150/1000
650/650 [==============================] - 413s 635ms/step - loss: 2.9131 - val_loss: 3.0868

Epoch 00150: val_loss di

650/650 [==============================] - 420s 646ms/step - loss: 2.8765 - val_loss: 3.0670

Epoch 00178: val_loss did not improve from 3.05489
Number of images: 45
Presicion: 0.5442
Recall: 0.4854
F1 score: 0.5131
Epoch 179/1000
650/650 [==============================] - 417s 642ms/step - loss: 2.8716 - val_loss: 3.0630

Epoch 00179: val_loss did not improve from 3.05489
Number of images: 45
Presicion: 0.5513
Recall: 0.4866
F1 score: 0.5169
Epoch 180/1000
650/650 [==============================] - 417s 642ms/step - loss: 2.8718 - val_loss: 3.0613

Epoch 00180: val_loss did not improve from 3.05489
Number of images: 45
Presicion: 0.5499
Recall: 0.4973
F1 score: 0.5223
Epoch 181/1000
650/650 [==============================] - 415s 638ms/step - loss: 2.8553 - val_loss: 3.0618

Epoch 00181: val_loss did not improve from 3.05489
Number of images: 45
Presicion: 0.5353
Recall: 0.4818
F1 score: 0.5071
Epoch 182/1000
650/650 [==============================] - 419s 645ms/step - loss: 2.8645 - 

In [12]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [13]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
# images_dir = data_path + 'udacity_driving_datasets'
# images_dir = data_path + 'pascal_dataset'
images_dir = data_path + 'images'

# Ground truth
# train_labels_filename = preprocess_path + '/udc.csv'
# train_labels_filename = preprocess_path + '/cic_pascal_train.csv'
# val_labels_filename   = preprocess_path + '/cic_pascal_val.csv'
# train_labels_filename = preprocess_path + '/pascal_train.csv'
# val_labels_filename   = preprocess_path + '/pascal_val.csv'
train_labels_filename = preprocess_path + '/cic_train.csv'
val_labels_filename   = preprocess_path + '/cic_val.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Optional: Convert the dataset into an HDF5 dataset. This will require more disk space, but will
# speed up the training. Doing this is not relevant in case you activated the `load_images_into_memory`
# option in the constructor, because in that cas the images are in memory already anyway. If you don't
# want to create HDF5 datasets, comment out the subsequent two function calls.

# train_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_train.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_val.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Loading images into memory: 100%|██████████| 45/45 [00:01<00:00, 27.77it/s]
Number of images in the training dataset:	   210
Number of images in the validation dataset:	    45


In [14]:
# 3: Set the batch size.
batch_size = 20 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	   210
Number of images in the validation dataset:	    45


In [15]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal_cic.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)

csv_logger = CSVLogger(filename=data_path + 'history/ssd300_adam_vgg13_bn/ssd300_vgg_13_pascal_cic.csv',
                       separator=',',
                       append=True)

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=11,
                               verbose=1)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.2,
                                         patience=9,
                                         verbose=1,
                                         min_delta=0.001,
                                         cooldown=0,
                                         min_lr=0.00001)

f1_callback = f1_call(0.20, 
                      0.45, 
                      200, 
                      normalize_coords, 
                      img_height, 
                      img_width, 
                      val_images_300,
                      label_val, 
                      (1, 8732, 14),
                      data_path + 'history/ssd300_adam_vgg13_bn/ssd300_vgg_13_pascal_cic_f1.csv')

# learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule,
#                                                 verbose=1)

# terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             early_stopping,
             reduce_learning_rate,
             f1_callback]

In [16]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 1000
steps_per_epoch = 22

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Epoch 1/1000
22/22 [==============================] - 63s 3s/step - loss: 4.2888 - val_loss: 7.8512

Epoch 00001: val_loss improved from inf to 7.85121, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal_cic.h5
Number of images: 45
Presicion: 0.0198
Recall: 0.3999
F1 score: 0.0377
Improve F1 score from -inf to 0.03765370799567672
Epoch 2/1000
22/22 [==============================] - 61s 3s/step - loss: 3.7467 - val_loss: 6.7193

Epoch 00002: val_loss improved from 7.85121 to 6.71928, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal_cic.h5
Number of images: 45
Presicion: 0.0602
Recall: 0.4211
F1 score: 0.1053
Improve F1 score from 0.03765370799567672 to 0.10533689237849736
Epoch 3/1000
22/22 [==============================] - 65s 3s/step - loss: 3.4730 - val_loss: 3.8798

Epoch 00003: val_loss improved from 6.71928 to 3.87982, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg1

22/22 [==============================] - 61s 3s/step - loss: 2.6298 - val_loss: 2.6599

Epoch 00030: val_loss improved from 2.73520 to 2.65991, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal_cic.h5
Number of images: 45
Presicion: 0.7035
Recall: 0.6207
F1 score: 0.6595
Improve F1 score from 0.6485113333076841 to 0.6595168201419787
Epoch 31/1000
22/22 [==============================] - 61s 3s/step - loss: 2.5017 - val_loss: 2.6445

Epoch 00031: val_loss improved from 2.65991 to 2.64448, saving model to /home/aldo/Documents/weights/models/ssd300_adam_vgg13_bn/ssd300_vgg_13_bn_pascal_cic.h5
Number of images: 45
Presicion: 0.7283
Recall: 0.6358
F1 score: 0.6789
Improve F1 score from 0.6595168201419787 to 0.6788904336954557
Epoch 32/1000
22/22 [==============================] - 58s 3s/step - loss: 2.4357 - val_loss: 2.5928

Epoch 00032: val_loss improved from 2.64448 to 2.59275, saving model to /home/aldo/Documents/weights/models/ssd300_adam_


Epoch 00059: val_loss did not improve from 2.37222
Number of images: 45
Presicion: 0.7256
Recall: 0.6291
F1 score: 0.6739
Epoch 60/1000
22/22 [==============================] - 60s 3s/step - loss: 2.0779 - val_loss: 2.4285

Epoch 00060: val_loss did not improve from 2.37222
Number of images: 45
Presicion: 0.7327
Recall: 0.6418
F1 score: 0.6842
Epoch 61/1000
22/22 [==============================] - 61s 3s/step - loss: 2.2392 - val_loss: 2.4359

Epoch 00061: val_loss did not improve from 2.37222
Number of images: 45
Presicion: 0.7151
Recall: 0.6848
F1 score: 0.6996
Epoch 62/1000
22/22 [==============================] - 60s 3s/step - loss: 2.2196 - val_loss: 2.3871

Epoch 00062: val_loss did not improve from 2.37222

Epoch 00062: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Number of images: 45
Presicion: 0.7217
Recall: 0.6791
F1 score: 0.6997
Epoch 63/1000
22/22 [==============================] - 62s 3s/step - loss: 2.2546 - val_loss: 2.3721

Epoch 00063: val_loss

22/22 [==============================] - 63s 3s/step - loss: 1.9538 - val_loss: 2.2821

Epoch 00089: val_loss did not improve from 2.25467
Number of images: 45
Presicion: 0.7474
Recall: 0.6526
F1 score: 0.6968
Epoch 00089: early stopping
